<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Bayesian-estimation-equivalent-of-Welch's-t-test" data-toc-modified-id="Bayesian-estimation-equivalent-of-Welch's-t-test-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Bayesian estimation equivalent of Welch's t-test</a></span><ul class="toc-item"><li><span><a href="#The-classic-Welch's-t-test" data-toc-modified-id="The-classic-Welch's-t-test-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>The classic Welch's t-test</a></span></li></ul></li><li><span><a href="#Bayesian-inference" data-toc-modified-id="Bayesian-inference-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Bayesian inference</a></span></li><li><span><a href="#Steps-of-Bayesian-data-analysis" data-toc-modified-id="Steps-of-Bayesian-data-analysis-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Steps of Bayesian data analysis</a></span></li><li><span><a href="#Step-1---Identify-the-relevant-data-for-question-under-investigation" data-toc-modified-id="Step-1---Identify-the-relevant-data-for-question-under-investigation-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Step 1 - Identify the relevant data for question under investigation</a></span><ul class="toc-item"><li><span><a href="#Study/data-description" data-toc-modified-id="Study/data-description-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Study/data description</a></span></li></ul></li><li><span><a href="#Step-2---Define-the-descriptive-statistical-model-\begin{align*}
y_{i}-&amp;\sim-Normal(\mu_{i},-\sigma_{k}-)-
\\-\mu_{i}-&amp;=\alpha_k
\\-\alpha_k-&amp;\sim-Normal()-\hspace{0.2cm}-for-\hspace{0.2cm}k\hspace{0.2cm}-1..2
\\-\sigma_{k}-&amp;\sim-Exponential()-\hspace{0.2cm}for-\hspace{0.5cm}k\hspace{0.2cm}-1..2
\end{align*}" data-toc-modified-id="Step-2---Define-the-descriptive-statistical-model-\begin{align*}
y_{i}-&amp;\sim-Normal(\mu_{i},-\sigma_{k}-)-
\\-\mu_{i}-&amp;=\alpha_k
\\-\alpha_k-&amp;\sim-Normal()-\hspace{0.2cm}-for-\hspace{0.2cm}k\hspace{0.2cm}-1..2
\\-\sigma_{k}-&amp;\sim-Exponential()-\hspace{0.2cm}for-\hspace{0.5cm}k\hspace{0.2cm}-1..2
\end{align*}-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Step 2 - Define the descriptive statistical model <div class="MathJax_Display" style="text-align: center;"></div><script type="math/tex; mode=display" id="MathJax-Element-2">\begin{align*}
y_{i} &\sim Normal(\mu_{i}, \sigma_{k} ) 
\\ \mu_{i} &=\alpha_k
\\ \alpha_k &\sim Normal() \hspace{0.2cm} for \hspace{0.2cm}k\hspace{0.2cm} 1..2
\\ \sigma_{k} &\sim Exponential() \hspace{0.2cm}for \hspace{0.5cm}k\hspace{0.2cm} 1..2
\end{align*}</script></a></span></li><li><span><a href="#Step-3---Specifying-priors" data-toc-modified-id="Step-3---Specifying-priors-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Step 3 - Specifying priors</a></span></li><li><span><a href="#Step-4---Use-Bayes-rule" data-toc-modified-id="Step-4---Use-Bayes-rule-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Step 4 - Use Bayes rule</a></span><ul class="toc-item"><li><span><a href="#Stan-model-of-Bayesian-Welch's-t-test" data-toc-modified-id="Stan-model-of-Bayesian-Welch's-t-test-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>Stan model of Bayesian Welch's t-test</a></span></li></ul></li><li><span><a href="#References" data-toc-modified-id="References-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>References</a></span></li></ul></div>

In [1]:
# Import analysis packages
%matplotlib inline
import pystan as ps
import numpy as np
import pandas as pd
import seaborn as sns
import arviz as az
import matplotlib.pyplot as plt
import scipy.stats as ss

In [2]:
from IPython.core.display import HTML as Center

Center(""" <style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style> """)

# Bayesian estimation equivalent of Welch's t-test

## The classic Welch's t-test

In order to understand the Welch's t-test from classical statistics it is important to first outline the student t-test (apologise to any readers who have already read the Bayesian estimation of between subkects t-test notebook but this is essentially all repetition).

The student t-test in formula form can be denoted as $$\large t = \frac{m_1-m_2}{\sqrt{\frac{S^2}{n_1}+\frac{S^2}{n_2}}}$$

$m_1 m_2 =$ the mean values for the two respective groups.

$n_1 n_2 =$ the sample size for the two respective groups.

and $S^2$ estimator for the varaince of the two groups being analysed 

$$\large S^2 = \frac{\sum(x-m_1)^2 + \sum(x-m_2)^2}{n_1 + n_2 -2}$$

with the denominator $n_1 + n_2-2$ being gthe calculation of degree of freedom for the test.

One of the assumptions of the student t-test results then from this aplication of a pooled estimator. That assumption being the homogeneity of variance (homoscedasticity).

Like all assumptions then the big question is for any analysis is are they reasonable within the application at hand. in recent years there has been arguements that the student t-test is inappropiate for research where variance between the two groups is unlikely (such as psychology/behavioural analysis), with even arguements that researchers should default to welch's t-test (Delacre, Lakens, & Leys, 2017), the impotance of a need to for default comes clearer with identifcation the prolific use of t-tests within psychology (Wetzels, et al. 2011) considering the variety of statistical tools.

Nevertheless, the welch's t-test is different from the student t-test because it can analyse two groups of data within classical statitics framework even in cases of heteroscedasticity.

The Welch's t-test in formula form can be denoted as $$\large t = \frac{m_1-m_2}{\sqrt{\frac{S_{1}^{2}}{n_1}+\frac{S_{2}^{2}}{n_2}}}$$

$S_{1}^{2} S_{2}^{2}=$ Standard deviation of the the respective two groups

The df are calculated as  $\large df = $

# Bayesian inference
<font size = "3"> Following the quick description of the classic welch's t-test above its important to keep in mind that  inference within Bayesian data analysis is all derived from the application of Bayes rule $P(\theta \mid y) = \large \frac{P(y \mid \theta) \, P(\theta)}{P(y)}$ and as such while the following description of the Bayesian model is an equivalent to the Welch's t-test, it is fundamentally different, because it uses fully probabilistic modelling and, therefore, any resulting infernces are not based on sampling distributions</font>
    
<font size = "1"> For a fuller description see the Practicing Bayesian statistics markdown file within the Github repository.</font>

# Steps of Bayesian data analysis

<font size = "3"> Kruscke (2015) offers a step by step formulation for how to conduct a Bayesian analysis:

1. Identify the relevant data for question under investigation.

2. Define the descriptive (mathematical) model for the data.

3. Specify the Priors for the model. If scientific research publication is the goal, the priors must be acceptable to a skeptical audience of peer reviewers. Much of this can be achieved using prior predictive checks to acsertain if the priors are reasonable.

4. Using Bayes rule estimate the posterior for the parameters of the model using the likelihood and priors. Then interpret the posterior for any statisitcal inferences.

5. Conduct model checks. i.e. Posterior predcitive checks, to idnefity if the model is reasonable and therfore also check if the infernces are reasonable in addition. </font> 


#  Step 1 - Identify the relevant data for question under investigation

In [ ]:
url = "https://raw.githubusercontent.com/ebrlab/Statistical-methods-for-research-workers-bayes-for-psychologists-and-neuroscientists/master/Data/James%20et%20al%202015%20Experiment%202%20Data%20Set.csv"

#Generare apndas data frame with the study data
df = pd.read_csv(url)
df

## Study/data description

# Step 2 - Define the descriptive statistical model \begin{align*}
y_{i} &\sim Normal(\mu_{i}, \sigma_{k} ) 
\\ \mu_{i} &=\alpha_k
\\ \alpha_k &\sim Normal() \hspace{0.2cm} for \hspace{0.2cm}k\hspace{0.2cm} 1..2
\\ \sigma_{k} &\sim Exponential() \hspace{0.2cm}for \hspace{0.5cm}k\hspace{0.2cm} 1..2
\end{align*} 


# Step 3 - Specifying priors

# Step 4 - Use Bayes rule

## Stan model of Bayesian Welch's t-test

In [20]:
Welch_t_model = """

data{

int<lower = 1> N; // number of data points
vector[N] y; // vector of data points in long format for both groups
int K; // number of groups
int<lower = 1, upper = K> x[N]; // array of interger valuesof the indicator variable for groups
int<lower = 0, upper = 1> onlyprior;
}

parameters{

// Model parameters to be estimated

 vector<lower = 0>[K] mu;// Bounded between 0 and 1 because as proportion of gaze scores cannot exceed those bounds
 vector<lower = 0>[K] sigma; //Standard deviation bounded at 0
 
}

model{

//priors
mu ~ normal(0,1);
sigma ~ exponential(1);

// Conditional stament whether to run prior predictive check or 
// full analysis
if (!onlyprior)
// Likliehood
y ~ normal(mu[x], sigma[x]);

}
generated quantities{
real diff = mu[1] - mu[2];
real Cohen_D = diff / (sigma[1] + sigma[2])/2;


real yrep[N];
  
// Generate data for posterior samples
  //for (i in 1:N) {
    yrep = normal_rng(mu[x], sigma[x]);
 //}
}
"""

In [21]:
# StanModel function can be called and be passed the model string specified above to compile into C++ code.
sm  = ps.StanModel(model_code = Welch_t_model)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_e9ff927b8f539eae8349ee663abf2f99 NOW.


In [ ]:
data = {'N': len(df),
        'y': df[""].values,
        'K': 2
        'x': x
        'onlyprior': 1}

# References
Delacre, M., Lakens, D., & Leys, C. (2017). Why psychologists should by default use Welch’s t-test instead of Student’s t-test. International Review of Social Psychology, 30(1).

Wetzels, R., Matzke, D., Lee, M. D., Rouder, J. N., Iverson, G. J., & Wagenmakers, E. J. (2011). Statistical evidence in experimental psychology: An empirical comparison using 855 t tests. Perspectives on Psychological Science, 6(3), 291-298.